In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/THESIS/rag-thesis'

/content/drive/MyDrive/THESIS/rag-thesis


In [ ]:
!pip install -U -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a version that satisfies the requirement tdqdm (

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import pymc as pm
import arviz as az
from forecasting_utils import *

In [4]:
df_processed = pd.read_csv('/content/drive/MyDrive/THESIS/rag-thesis/data/processed_FRED_data.csv')
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
df_processed.head()

,observation_date,GDP_log,XTIMVA01USM664S_log,XTEXVA01USM664S_log,GPDI_log,RPI_log,INDPRO_log,BUSLOANS_log,NONREVSL_log,UNRATE_diff,...,GS1_diff,AAA_diff,IRLTLT01USM156N_diff,M1V,CUMFNS,CLF16OV,B235RC1Q027SBEA,ROWFDIQ027S,CPILFESL_inflation,PCEPI_inflation
0,1960-04-01,-1.889106,-1.937830,-1.900735,-1.933337,-1.969102,-2.223121,-1.891318,-1.855541,0.143477,...,-0.991366,-0.273408,-0.492671,-1.208557,0.410447,-1.730502,-0.917074,-0.751348,-1.084373,-0.461731
1,1960-07-01,-1.882088,-1.966669,-1.884644,-1.940230,-1.964744,-2.260716,-1.847751,-1.841514,0.420758,...,-1.133084,-0.381349,-0.925014,-1.210446,-0.080174,-1.724082,-0.920047,-0.750873,-1.585535,-0.688792
2,1960-10-01,-1.890482,-2.003349,-1.890457,-2.047059,-1.966695,-2.316039,-1.805262,-1.830178,1.113960,...,-0.112715,0.023429,0.134228,-1.227453,-0.698059,-1.713532,-0.921271,-0.752034,-0.573631,-0.579965
3,1961-01-01,-1.882994,-2.008331,-1.877366,-2.021036,-1.951916,-2.350629,-1.773634,-1.821522,0.698039,...,-0.169402,-0.138482,-0.211647,-1.220367,-1.108528,-1.704369,-0.921796,-0.750767,-1.086546,-0.984947
4,1961-04-01,-1.866870,-2.003468,-1.911936,-1.962365,-1.926400,-2.262478,-1.750164,-1.825352,0.282117,...,0.099862,0.023429,0.004525,-1.198636,-0.596247,-1.704078,-0.922670,-0.750318,-0.916503,-1.296390


### Forecasting Benchmark Implementation:

To conduct the sampling of RMSEs for each variable under each forecast horizion for each model, we will do the following:

- Starting with forecast h=1 and a a 70-30 training-test split, we will iterate over each variable and employ forecasting with horizion h. In other words, starting at quarter i into the time series data, we will forcast the next h quarters. At the time time, we use a univariate AR(1) model to forecast the same next h quarters.

- We will then calculate the MSE beteween the actual and forecasted values for each variable. We do the same between the AR(1) predicted and actual values. This is one obvervation.

- We then forecast the next h quarters starting at quarter i+1 into the time series and calculate the MSE of the forecast for each variable relative to the AR(1). This is the second observation. in other words, we have shifted 1 quarter out of the test set and into the training set.

- We repeat this process until 95% of the data is in the training set.

- From these obvervations, we sum up all mse's for the model, sum up all the mses of the AR(1), and then we take the square root of the mdoel forecasting error and the AR(1) error. This is the RMSFE. We then take the mean, std, min, max, q25, q75, and iqr. These are the summary statistics for the models RMSFE relative to AR(1) with horizion h.

- We then repeat the process for h=3, h=6, and h=12.

- We then repeat the process for each model.

**For TimeGPT, we will shift the training-test split by 2% of the data each iteration due to API limits

### AR(1) Benchmark

In [5]:
# Example usage:
target_variable = 'GDP_log'
# Try different AR orders
train_data, test_data, _ = test_train_split(df_processed, test_size=0.3)
print(run_ar_benchmark(train_data, test_data, target_variable, ar_order=1, horizon=3))

(np.float64(0.001151469306476379), np.float64(0.10475295593796632))


### Bayesian Vector Autoregression (BVAR)

In [6]:
df_processed = df_processed.select_dtypes(include=[np.number])
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
bvar_results = {
    h:{
        'RMSE': [],
        'cum_RMSE': {var:[] for var in variables}
    } for h in ["h=1", "h=3", "h=6", "h=12"]
}

for h in [1,3,6,12]:
    print("Forecasting on horizon: " + str(h))
    test_size = 0.3
    train_size = int(len(df_processed)*(1-test_size))
    train_data = df_processed.iloc[:train_size]
    test_data = df_processed.iloc[train_size:]

    current_cum_sums ={var:0 for var in variables}
    rmses = {var:[] for var in variables}
    ar_rmses = {var:[] for var in variables}

    while len(test_data) > 13:

        rmse_dict, _ = fit_evaluate_bvar_pymc(train_data, test_data, horizon=h, lag_order=2, num_samples=200)
        for var in variables:
            ar_rmse, _ = run_ar_benchmark(train_data, test_data, var, ar_order=1, horizon=h)
            current_cum_sums[var] += rmse_dict[var]
            bvar_results["h="+str(h)]['cum_RMSE'][var].append(current_cum_sums[var])
            rmses[var].append(rmse_dict[var])
            ar_rmses[var].append(ar_rmse)


        train_size += 3
        train_data = df_processed.iloc[:train_size]
        test_data = df_processed.iloc[train_size:]

    for var in variables:
        bvar_results["h="+str(h)]['RMSE'].append(np.sum(rmses[var])/np.sum(ar_rmses[var]))

print(bvar_results)

Forecasting on horizon: 1


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


...

Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


{'h=1': {'RMSE': [np.float64(1.2575354631550404), np.float64(1.2612386065549617), np.float64(1.1888927532108888), np.float64(0.6225513511468176), np.float64(1.0846246319763455), np.float64(0.9779041316592991), np.float64(0.7353147127409809), np.float64(1.8534342411894842), np.float64(1.0759567504445189), np.float64(2.6088922475737304), np.float64(2.0581741275815815), np.float64(0.9373157822684454), np.float64(0.930420787341747), np.float64(1.0322147007203102), np.float64(0.9062108256758649), np.float64(0.8896721316536296), np.float64(1.0663099052481277), np.float64(0.9531274573934597), np.float64(1.3307927812731002), np.float64(1.0129550056747967)], 'cum_RMSE': {'GDP_log': [np.float64(0.009463726907510517), np.float64(0.021959197409233244), np.float64(0.030646336919616712), np.float64(0.032915877027237306), np.float64(0.03377293691507621), np.float64(0.05123412411323047), np.float64(0.057513689744956076), np.float64(0.06321357241661607), np.float64(0.06739958101493149), np.float64(0.07

In [7]:
for h in bvar_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(bvar_results[h]['RMSE'])
    #rmse_array = rmse_array[rmse_array <= 8]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

Horizon h=1:
Mean: 1.1891769197241566
Std:  0.46173676079097314
Min:  0.6225513511468176
Max:  2.6088922475737304
Q25:  0.9355920335367708
Median:  1.049262302984219
Q75:  1.2584612490050207
IQR:  0.32286921546824987
Horizon h=3:
Mean: 1.3627662228883985
Std:  0.39222049709816686
Min:  0.9464924964879319
Max:  2.5429157116639773
Q25:  1.0860690325833773
Median:  1.2016576425814443
Q75:  1.5456999271976994
IQR:  0.45963089461432216
Horizon h=6:
Mean: 1.4808805902972555
Std:  0.4270060395687694
Min:  0.9162185272700611
Max:  2.4289995964195916
Q25:  1.1786163786115307
Median:  1.3196961764429542
Q75:  1.7176546849141885
IQR:  0.5390383063026578
Horizon h=12:
Mean: 1.6559881174074094
Std:  0.6722376110400714
Min:  0.8255794278234918
Max:  3.1342980759618557
Q25:  1.1775318769100738
Median:  1.4710219297358091
Q75:  1.9109255705583714
IQR:  0.7333936936482977


In [8]:
# Save to JSON
import json
with open('results/bvar_results.json', 'w') as f:
    json.dump(bvar_results, f,indent=4,)

### ARIMA

In [ ]:
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
arima_results = {
    h:{
        'RMSE': [],
        'cum_RMSE': {var:[] for var in variables}
    } for h in ["h=1", "h=3", "h=6", "h=12"]
}

for h in [1,3,6,12]:
    print("Forecasting on horizon: " + str(h))
    test_size = 0.3
    train_size = int(len(df_processed)*(1-test_size))
    train_data = df_processed.iloc[:train_size]
    test_data = df_processed.iloc[train_size:]

    current_cum_sums ={var:0 for var in variables}
    rmses = {var:[] for var in variables}
    ar_rmses = {var:[] for var in variables}

    while len(test_data) > 13:
        for var in variables:

            rmse, _ = fit_evaluate_arima(var, train_data, test_data, h)
            ar_rmse, _ = run_ar_benchmark(train_data, test_data, var, ar_order=1, horizon=h)

            current_cum_sums[var] += rmse
            #print(current_cum_sums[var])
            arima_results["h="+str(h)]['cum_RMSE'][var].append(current_cum_sums[var])
            rmses[var].append(rmse)
            ar_rmses[var].append(ar_rmse)

        train_size += 1
        train_data = df_processed.iloc[:train_size]
        test_data = df_processed.iloc[train_size:]


    for var in variables:
        arima_results["h="+str(h)]['RMSE'].append(np.sum(rmses[var])/np.sum(ar_rmses[var]))



Forecasting on horizon: 1
Forecasting on horizon: 3
Forecasting on horizon: 6
Forecasting on horizon: 12


In [ ]:
for h in arima_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(arima_results[h]['RMSE'])
    #rmse_array = rmse_array[rmse_array <= 5]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

NameError: name 'arima_results' is not defined

In [ ]:
# Save to JSON
import json
with open('results/arima_results.json', 'w') as f:
    json.dump(arima_results, f,indent=4,)

### TimeGPT

Multiseries Forecast:
https://docs.nixtla.io/docs/capabilities-forecast-multiple_series_forecasting



In [ ]:
pip install -Uqq nixtla datasetsforecast utilsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
from nixtla import NixtlaClient
from datasetsforecast.long_horizon import LongHorizon
from utilsforecast.losses import mae

nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'nixak-Q6bqxtnGbhVX0aqgWPvnwwnAIeyRk4dSJJELRDUBnoYwYnsaUBh1rHTpzhgdX5Bdp9wUCmpyB83SYhIo'
)

In [ ]:
# Prepare data for TimeGPT by reshaping from wide to long format
def prepare_data_for_timegpt(df):
    # Create a copy to avoid modifying the original
    df = df.copy()

    # Rename the date column to 'ds'
    df = df.rename(columns={'observation_date': 'ds'})

    # Get list of numeric columns (these will be our variables to forecast)
    numeric_columns = df.select_dtypes(include=[np.number]).columns

    # Melt the dataframe to convert from wide to long format
    df_long = pd.melt(
        df,
        id_vars=['ds'],
        value_vars=numeric_columns,
        var_name='unique_id',
        value_name='y'
    )

    # Sort by unique_id and date
    df_long = df_long.sort_values(['unique_id', 'ds'])

    return df_long

# Apply the transformation
df_processed = pd.read_csv('/content/drive/MyDrive/THESIS/rag-thesis/data/processed_FRED_data.csv')
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
df_timegpt = prepare_data_for_timegpt(df_processed)

# Display the first few rows of the transformed data
print("Shape of transformed data:", df_timegpt.shape)
print("\nFirst few rows of transformed data:")
print(df_timegpt.head())

# Display unique variables being forecast
print("\nUnique variables being forecast:")
print(df_timegpt['unique_id'].unique())

Shape of transformed data: (5160, 3)

First few rows of transformed data:
              ds unique_id         y
2838  1960-04-01  AAA_diff -0.273408
2839  1960-07-01  AAA_diff -0.381349
2840  1960-10-01  AAA_diff  0.023429
2841  1961-01-01  AAA_diff -0.138482
2842  1961-04-01  AAA_diff  0.023429

Unique variables being forecast:
['AAA_diff' 'B235RC1Q027SBEA' 'BUSLOANS_log' 'CLF16OV'
 'CPILFESL_inflation' 'CUMFNS' 'DFF_diff' 'GDP_log' 'GPDI_log' 'GS1_diff'
 'INDPRO_log' 'IRLTLT01USM156N_diff' 'M1V' 'NONREVSL_log'
 'PCEPI_inflation' 'ROWFDIQ027S' 'RPI_log' 'UNRATE_diff'
 'XTEXVA01USM664S_log' 'XTIMVA01USM664S_log']


In [ ]:
def run_timegpt(df_long, df_transformed, ratio=0.7):
    # Horizons to evaluate
    horizons = [1, 3, 6, 12]
    results = {
        h: {
            'RMSE': [],
            'cum_RMSE': {var:[] for var in variables}
        } for h in ["h=1", "h=3", "h=6", "h=12"]
    }

    for h in horizons:
        print(f"Running for horizon {h}...")
        # Create a copy to avoid modifying the original
        df = df_long.copy()

        # Split data into train and test
        split_idx = 180
        train_dates = sorted(df['ds'].unique())[:split_idx]

        train_data = df[df['ds'].isin(train_dates)]
        test_data = df[~df['ds'].isin(train_dates)]

        # Test train split for AR data (wide format)
        train_data_ar = df_processed.iloc[:split_idx]
        test_data_ar = df_processed.iloc[split_idx:]

        # Calculate RMSE and statistics for each variable
        current_cum_sums ={var:0 for var in variables}
        rmses = {var:[] for var in variables}  # Store all RMSE values for this horizon
        ar_rmses = {var:[] for var in variables}

        while split_idx < 245:
            # Generate forecast
            forecast_df = nixtla_client.forecast(
                df=train_data,
                h=h,
                level=[80, 90]
            )
            # Calculate metrics for each unique_id (variable)
            for var in df['unique_id'].unique():
                # Get actual values for this horizon and variable
                var_test = test_data[test_data['unique_id'] == var]['y'].iloc[:h].values
                # Get predictions for this variable
                var_pred = forecast_df[forecast_df['unique_id'] == var]['TimeGPT'].values

                # Calculate RMSE
                rmse = np.sqrt(mean_squared_error(var_test, var_pred))
                ar1_rmse, _ = run_ar_benchmark(train_data_ar, test_data_ar, var, ar_order=1, horizon=h)

                # Track metrics
                current_cum_sums[var] += rmse
                results["h="+str(h)]['cum_RMSE'][var].append(current_cum_sums[var])
                rmses[var].append(rmse)
                ar_rmses[var].append(ar1_rmse)

            # Shift test-train split by 1 timestamp
            split_idx += 1
            train_dates = sorted(df['ds'].unique())[:split_idx]
            train_data = df[df['ds'].isin(train_dates)]
            test_data = df[~df['ds'].isin(train_dates)]

            train_data_ar = df_processed.iloc[:split_idx]
            test_data_ar = df_processed.iloc[split_idx:]

        for var in variables:
            # Compute root mean square error for each variable
            results["h="+str(h)]['RMSE'].append(np.sum(rmses[var])/np.sum(ar_rmses[var]))

    return results

# Example usage:
timegpt_results = run_timegpt(df_timegpt, df_processed)

# Print results
print("\nDetailed RMSE for each variable and horizon:")
print(timegpt_results)

Running for horizon 1...
Running for horizon 3...
Running for horizon 6...
Running for horizon 12...

Detailed RMSE for each variable and horizon:
{'h=1': {'RMSE': [np.float64(1.4020648686642), np.float64(1.3175076586874603), np.float64(1.1228547636047104), np.float64(1.1027464535434683), np.float64(1.0725602459781458), np.float64(1.0364026355676461), np.float64(0.7760996623609011), np.float64(1.6701888061927341), np.float64(0.8569757706996182), np.float64(1.266431343922709), np.float64(1.2885783673538398), np.float64(1.1457496450361486), np.float64(1.1771660128049541), np.float64(1.00139781898111), np.float64(1.1361252801740873), np.float64(1.0495630778195804), np.float64(1.1789495397996), np.float64(1.007493549268818), np.float64(0.9155561018301117), np.float64(1.0214788129639683)], 'cum_RMSE': {'GDP_log': [np.float64(0.010782114114395669), np.float64(0.01638111657690633), np.float64(0.022995956276935403), np.float64(0.02376395728093461), np.float64(0.032803407668622775), np.float64(

In [ ]:
for h in timegpt_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(timegpt_results[h]['RMSE'])
    #rmse_array = rmse_array[rmse_array <= 5]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

Horizon h=1:
Mean: 1.1272945207626908
Std:  0.19466941258261133
Min:  0.7760996623609011
Max:  1.6701888061927341
Q25:  1.0179824970401807
Median:  1.1128006085740894
Q75:  1.2008199908303772
IQR:  0.18283749379019643
Horizon h=3:
Mean: 1.0879459263441775
Std:  0.14220982637339677
Min:  0.7511806803223179
Max:  1.3237546363923107
Q25:  1.0027485424039584
Median:  1.0863611607895693
Q75:  1.194430679563871
IQR:  0.19168213715991267
Horizon h=6:
Mean: 1.100126287834025
Std:  0.15626256633930807
Min:  0.8313361431623699
Max:  1.4471599557736474
Q25:  1.0039036040431721
Median:  1.0850605036147125
Q75:  1.196066731286656
IQR:  0.19216312724348383
Horizon h=12:
Mean: 1.1390087461232832
Std:  0.18755166523199146
Min:  0.7545416206767542
Max:  1.562294984210576
Q25:  1.046570161398962
Median:  1.1589710648597684
Q75:  1.2101239062688707
IQR:  0.1635537448699087


In [ ]:
# Save to JSON
import json
with open('results/timegpt_results.json', 'w') as f:
    json.dump(timegpt_results, f,indent=4,)